## **1. OBJECTIFS**

Nous voulons évaluer rigoureusement la robustesse de notre modélisation et notre implémentation au-delà de sa seule validité théorique et sa faisabilité technique. Dans quelles conditions notre approche produit-elle des solutions de bonne qualité en un temps de calcul raisonnable ? Avec quelle variabilité ? Comment se comporte l'algorithme au cours de sa recherche, et qu'est ce qui explique son succès où son échec ?

Cette étude vise ainsi à établir la viabilité de notre solution à travers une **évaluation empirique** complète, en se reposant sur des benchmarks standardisés, des analyses statistiques et une lecture comportementale. Elle soulève les questions suivantes :  
- Qualité : l'écart relatif à un optimum connu et le respect strict des contraintes
- Robustesse : la variabilité et la part de l'aléatoire entre différentes exécutions des algorithmes, ainsi que la sensibilité aux paramètres et aux changements
- Scalabilité : la façon dont le temps d'exécution et de l'écart à l'optimum évoluent lorsque la taille des données d'entrée et la difficulté des instances croissent.
- Comportement : la trajectoire et l'équilibre entre l'intensification et la diversification

Pour y répondre, nous mettrons en œuvre un **plan d'expériences** (Design of Experiments - **DoE**). Cette approche méthodologique permet d'étudier l'effet de multiples facteurs (les paramètres de notre algorithme, les données d'une instance) sur les indicateurs de performance. Plutôt que de faire varier un seul paramètre à la fois, un plan d'expériences nous permettra d'identifier efficacement les facteurs les plus influents, de quantifier leur impact et de déceler d'éventuelles interactions entre eux.

Afin de garantir toute reproductibilité et transparence, nous fourniront le plus de détail possible sur l'environnement utilisé, les graines d'aléatoire, les temps limites, les paramètres, les fichiers d'instances, les décisions prises ou encore les benchmarks.

## **2. PROTOCOLE**

#### **2.1. BENCHMARKS**

Les performances de notre algorithme sont évaluées sur un ensemble d'instances de référence, issus de trois **benchmarks** reconnus dans la littérature pour notre problème.

Un **benchmark** est un ensemble d'instances représentatives, pour lesquelles il existe à chaque fois une **meilleure solution connue** (Best Know Solution, **BKS**), c'est-à-dire la meilleure solution trouvée à ce jour par la communauté scientifique. Les BKS ne sont pas systématiquement les solutions optimales, mais elles permettent déjà de se situer, et ce par rapport aux meilleurs algorithmes existants.

Nous avons sélectionnés 3 benchmarks, tous reconnus dans la littérature pour notre problème et respectivement basés sur les instances issues de la bibliothèque **VRPLIB**, des instances **SOLOMON**, introduites par Solomon (1987), et des instances **HG**, introduites par Hommbeger et Gehring (1999).

#### **2.2. INDICATEURS DE PERFORMANCE**

Pour évaluer les résultats de notre algorithme, nous nous baserons sur plusieurs indicateurs.
Pour mesurer la qualité, nous prendrons en compte la valeur de la fonction objectif, c'est-à-dire le coût total, et nous calculerons grâce à celle-ci l'écart relatif par rapport à la meilleure solution connue selon l'instance. Cet **écart à l'optimum**, exprimé en pourcentage, s'obtient avec la formule :
$$
Écart = \frac{résultat - BKS}{BKS} \times 100
$$
Pour s'assurer de la robustesse, chaque instance sera exécutée plusieurs fois. Nous analyserons ensuite la moyenne, l'écart-type et les extremums obtenus sur ces exécutions. En effet, de par la nature stochastique de notre méta-heuristique, on ne peut se contenter d'une seule exécution.

## **3. PLAN D'EXPÉRIENCE**

Pour calibrer les **facteurs** de notre méta-heuristique, c'est-à-dire les paramètres de l'algorithme qui influent sur l'intensité des mécanismes implémentés, nous pouvons naïvement faire varier un seul paramètre à la fois jusqu'à obtenir un bon résultat. Cependant, cette méthode est à la fois extrêmement coûteuse en temps de calcul et incapable de détecter les interactions entre les paramètres. Il est fréquent que l'effet d'un paramètre dépende de la valeur d'un autre. C'est pourquoi nous mettons en place un plan d'expérience, avec l'objectif de trouver une configuration de facteurs globalement robuste et performante.

#### **3.1. IDENTIFICATION**

La première étape consiste à lister les paramètres de notre méta-heuristique qui semblent avoir un impact sur sa performance et définir pour chacune un niveau . Nous avons identifié trois facteurs principaux, tous quantitatifs, qui régissent l'équilibre entre diversification et intensification, la vitesse d'adaptation et le critère d'acceptation des solutions.

Bien qu'il existe d'autres facteurs, nous nous sommes limités aux principaux, car le nombre d'expériences croît souvent exponentiellement selon le nombre facteurs et de niveaux. Notre sélection s'appuie sur notre propre analyse préliminaire de leur influence potentielle et sur les pratiques courantes dans la [littérature scientifique¹](#1).

Nous avons :
- Le taux $\alpha$ de refroidissement, qui permet d'accepter des solutions de moins bonne qualité avec une probabilité qui diminue au fil du temps, et ce pour éviter les blocages. **Niveaux :** 0.95, 0.99, 0.9999
- Le degré de destruction, qui contrôle le nombre de clients à retirer de la solution courante. Nous savons déjà qu'un faible degré favorise l'intensification tandis qu'un degré élevé favorise la diversification. **Niveaux :** 10, 30, 50 (%)
- Le facteur $\rho$ de réduction pour l'adaptation des poids, qui contrôle la vitesse à laquelle les poids des opérateurs de destruction et de répartition s'adaptent en fonction de leur performance passée. **Niveaux :** 0.5, 0.7, 0.9

Les autres paramètres, tels que la température initiale ou les scores d'attribution de récompense seront fixés à des valeurs standards pour ne pas complexifier excessivement l'analyse.

#### **3.2.TOPOLOGIE**

Il existe de nombreux **designs** pour concevoir son plan.

D'abord, il faut noter que la nature stochastique de notre algorithme nous impose répéter de plusieurs fois la même expérience. Nous notons ce nombre de répétitions $r$.

On utilie généralement un **plan factoriel complet** (FullFD). Un tel plan d'expérience teste toutes les combinaisons possibles. Pour $k$ facteurs de chacun $n$ niveaux, le nombre $N$ d'expériences à mener est égal à $n^{k}\times r$.

Le **plan factoriel fractionnaire** (FracFD) ne teste qu'une fraction des combinaisons possibles tout en conservant de l'information sur les effets principaux entre les facteurs. Il n'y a plus que $n^{k-p}$ expériences à mener, avec $p$ le fractionnement. Attention, plus $p$ est grand, plus le risque de confusion entre effets est élevé.

Pour une calibration plus fine, il existe aussi le **plan central composite** (CCD) qui capture plus les effets quadratiques pour un nombre d'expériences égal à $2^{k}+2k+n_{c}$ avec $n_{c}$ le nombre de points centraux.

Le plan factoriel complet devient pertinent lorsque plus de 4 à 6 facteurs entrent en jeu. Avec 3 facteurs, nous pouvons utiliser un plan complet. Tout de même limité par nos ressources, nous choisissons un $r$ minimal de $3$. Un $r$ inférieur ne serais pas rigoureux, puisque 2 expériences sont pas suffisantes pour constituer une moyenne et un écart intéressant. Le choix d'un CDD n'est pas non plus pertinent, puisque nous cherchons avant tout à réaliser une première calibration.

Avec un tel plan, nous devrons mener 81 expériences ($N=3^{3}\times3$).

#### **3.3. EXÉCUTION**

Notre méta-heuristique prend en paramètre le taux de refroidissement avec la variable ``cooling_rate``. Le degré de construction étant borné par les variables ``min_destroy_size`` et ``max_destroy_size``, nous considérons notre degré de destruction comme le milieu de cette intervalle et nous déterminons une largeur $w$ relative au degré, puisque l'impact de ce dernier sur la performance n'est pas linéaire. ``weight_decay`` représente notre facteur de réduction.

In [1]:
import sys
from pathlib import Path

project_root = Path("/collab/livrable/L2/framework").resolve()
if not project_root.exists():
    raise FileNotFoundError(f"Le chemin {project_root} n'existe pas.")
sys.path.insert(0, str(project_root))

added = {str(project_root)}
for p in project_root.rglob('*'):
    try:
        if p.is_dir() and not p.name.startswith('.'):
            sp = str(p.resolve())
            if sp not in added:
                sys.path.insert(0, sp)
                added.add(sp)
    except (OSError, PermissionError):
        continue

print(f"{len(added)} dossiers ajoutés au sys.path")

from framework.solvermanager.solvermanager_exe import SolverManager
from framework.solvermanager.solvers.alns import ALNSConfig

manager = SolverManager(
    data_dir="/data",
    results_dir="/results"
)

10 dossiers ajoutés au sys.path


In [2]:
cooling_rates = [0.95, 0.99, 0.9999]
min_destroy_sizes = [5, 20, 35]
max_destroy_sizes = [15, 40, 65]
weight_decays = [0.5, 0.7, 0.9]
seeds = [40,41,42]

runs = []


for i in range(3): # cooling rate 
    for j in range(3): # destroy level
        for k in range(3): # weight decay 
            for l in range(3): # try with different seeds
                config = ALNSConfig(
                    destroy_operators=['random', 'worst', 'shaw', 'route'],
                    repair_operators=['greedy', 'regret2', 'regret3'],
                    min_destroy_size=min_destroy_sizes[j],
                    max_destroy_size=max_destroy_sizes[j],
                    weight_decay=weight_decays[k],
                    weight_update_interval=50,
                    score_new_best=50.0,
                    score_better=10.0, score_accepted=2.0, score_rejected=0.0,
                    initial_temperature=1500.0,
                    cooling_rate=cooling_rates[0],
                    min_temperature=0.001,
                    shaw_removal_randomness=8.0,
                    max_time=120.0,
                    max_iterations_no_improvement=2000,
                    verbose=True,
                    seed=seeds[l]
                )
                results = manager.run_experiment(
                    instance_name="C101",
                    solver_name="alns",
                    config=config,
                    constructor="savings_parallel",
                    seed=seeds[l],
                    force_recompute = False,
                )
                runs.append({
                    "cost":results.cost,
                    "cooling_rate":cooling_rates[i],
                    "min_destroy_size":min_destroy_sizes[j],
                    "max_destroy_size":max_destroy_sizes[j],
                    "weight_decay":weight_decays[k],
                    "seed":seeds[l]})


Experiment: C101 + alns
[1/6] Loading instance 'C101'...
      → 100 clients, capacity=200
[2/6] Creating evaluator...
[LOAD] Found existing results — loaded


Experiment: C101 + alns
[1/6] Loading instance 'C101'...
      → 100 clients, capacity=200
[2/6] Creating evaluator...
[LOAD] Found existing results — loaded


Experiment: C101 + alns
[1/6] Loading instance 'C101'...
      → 100 clients, capacity=200
[2/6] Creating evaluator...
[LOAD] Found existing results — loaded


Experiment: C101 + alns
[1/6] Loading instance 'C101'...
      → 100 clients, capacity=200
[2/6] Creating evaluator...
[LOAD] Found existing results — loaded


Experiment: C101 + alns
[1/6] Loading instance 'C101'...
      → 100 clients, capacity=200
[2/6] Creating evaluator...
[LOAD] Found existing results — loaded


Experiment: C101 + alns
[1/6] Loading instance 'C101'...
      → 100 clients, capacity=200
[2/6] Creating evaluator...
[LOAD] Found existing results — loaded


Experiment: C101 + alns
[1/6] Loading i

In [3]:
print(len(runs))
print(runs[0])

81
{'cost': 827.2999999999998, 'cooling_rate': 0.95, 'min_destroy_size': 5, 'max_destroy_size': 15, 'weight_decay': 0.5, 'seed': 40}


#### **3.4. ANALYSE DE VARIANCE**

L'analyse de variance évalue l'influence des facteurs contrôlés dans un plan d'expérience. Elle permet plus précisémment de déterminer si les différences observées entre les résultats des expériences sont significativement attribuables aux facteurs étudiés. Heuresement, il existe la bibliothèque python ``statsmodels`` qui permet d'appliquer rapidement cette méthode. Pour traiter nos données, nous utiliserons ``pandas``.

In [4]:
import sys
!{sys.executable} -m pip install pandas statsmodels

Defaulting to user installation because normal site-packages is not writeable


In [5]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
import scipy.stats as stats
import matplotlib.pyplot as plt

Dans un premier temps, nous devons préparer les données sous forme d'un tableau. Chaque ligne représente une expérience unique, avec les valeurs des facteurs et la métrique de performance. Le tableau doit prendre en compte les variabilités, c'est-à-dire nos 81 expériences.

In [6]:
data = pd.DataFrame(runs)
data['dod'] = (data['min_destroy_size'] + data['max_destroy_size']) / 2
data = data.rename(columns={
    'cooling_rate': 'α',
    'weight_decay': 'ρ',
})

Ensuite, nous réalisons l'analyse en elle-même. Grâce à la bibliothèque, il suffit de définir une formule qui décrit le modèle statistique, incluant les facteurs et leurs interactions, puis de l'ajuster.

In [7]:
# 1. Spécification du modèle et ajustement
formula = "cost ~ C(α) * C(dod) * C(ρ)"
model = ols(formula, data=data).fit()

# 2. Affichage des résultats
anova_table = sm.stats.anova_lm(model, typ=2)
print("Tableau ANOVA :")
print(anova_table)

Tableau ANOVA :
                        sum_sq    df             F    PR(>F)
C(α)              6.908946e-24   2.0  1.840586e-26  1.000000
C(dod)            6.458600e+02   2.0  1.720611e+00  0.188622
C(ρ)              2.086580e+03   2.0  5.558778e+00  0.006379
C(α):C(dod)       2.123088e-24   4.0  2.828019e-27  1.000000
C(α):C(ρ)         6.235892e-25   4.0  8.306401e-28  1.000000
C(dod):C(ρ)       1.291720e+03   4.0  1.720611e+00  0.158868
C(α):C(dod):C(ρ)  1.381896e-24   8.0  9.203642e-28  1.000000
Residual          1.013490e+04  54.0           NaN       NaN


Ce tableau ANOVA résume l’influence de chaque facteur et de leurs interactions sur la performance de l’algorithme. Pour chaque terme, on retrouve la somme des carrés (sum_sq), les degrés de liberté (df), la statistique F et la valeur p associée.

ρ est le seul levier qui semble vraiment compter ici. Les autres facteurs, dans les plages testées, ne montrent pas d’effet significatif, et les interactions sont absentes. Cela simplifie l’optimisation : on peut se concentrer sur l’ajustement de ρ, en gardant à l’esprit que la variabilité résiduelle suggère qu’il reste peut-être des aspects à explorer (autres paramètres, heuristiques, ou plages de valeurs plus larges).

Cette observation s'implifie l’optimisation : on peut se concentrer sur l’ajustement fin de ρ pour améliorer les performances. Cependant, il est important de noter que la **variabilité résiduelle** reste élevée, ce qui suggère que d’autres aspects non pris en compte dans cette analyse pourraient jouer un rôle.

## **4. ANALYSE**

## **5. BIBLIOGRAPHIE**

<a id="1">1</a> — ALNS, ULM University (2014), section 5.4 : https://d-nb.info/1072464683/34